# Tutorial: Using `dual_autodiff_x`

This notebook only uses dual_autodiff_x


In [1]:
# Importing the packages
import math
import dual_autodiff_x.dual as dfx

# Let's create dual numbers using both implementations
x = dfx.Dual(2.0, 1.0)    # Using dual_autodiff
y = dfx.Dual(3.0, 0.5)   # Using dual_autodiff_x

print("Created dual numbers:")
print("x =", x)
print("y =", y)


Created dual numbers:
x = Dual(real=2.0, dual=1.0)
y = Dual(real=3.0, dual=0.5)


## Basic Arithmetic with Dual Numbers

Dual numbers can be added, subtracted, multiplied, and divided just like regular numbers. The real part behaves like a normal number, and the dual part tracks the derivative information.

- **Addition**: (a + bε) + (c + dε) = (a+c) + (b+d)ε
- **Multiplication**: (a + bε) * (c + dε) = (a*c) + (a*d + b*c)ε

We can also add and multiply dual numbers by scalars. Let’s try out these operations and see the results.


In [2]:
# Basic operations
addition = x + y
subtraction = x - y
multiplication = x * y
division = x / y

print("Addition:        ", addition)
print("Subtraction:     ", subtraction)
print("Multiplication:  ", multiplication)
print("Division:        ", division)

# Also demonstrate addition with a scalar
add_scalar = x + 5.0
print("Add scalar (x+5):", add_scalar)


Addition:         Dual(real=5.0, dual=1.5)
Subtraction:      Dual(real=-1.0, dual=0.5)
Multiplication:   Dual(real=6.0, dual=4.0)
Division:         Dual(real=0.6666666666666666, dual=0.2222222222222222)
Add scalar (x+5): Dual(real=7.0, dual=1.0)


## Computing Derivatives with Dual Numbers

One of the main advantages of using dual numbers is their ability to compute derivatives automatically. If we represent `x` as `x = a + ε`, where `ε` is the dual unit, then for a function `f(x)`, substituting `x = a + 1*ε` gives us:

- `f(x).real` = f(a)
- `f(x).dual` = f'(a)

To illustrate this, consider the function:

$$
f(x) = \sin(x) + x^2
$$

We know the analytical derivative is:

$$
f'(x) = \cos(x) + 2x
$$

We will:
1. Compute the derivative using dual numbers.
2. Compute the analytical derivative using `math.cos` and compare.


In [3]:
def f(u):
    # f(x) = sin(x) + x^2
    return u.sin() + u**2

# Let's pick a point to evaluate, say x = 1.5
x_val = 1.5

# Create a dual number with dual part = 1.0 to track the derivative
x_dual = dfx.Dual(x_val, 1.0)
result = f(x_dual)

# Extract the computed derivative from dual number
dual_derivative = result.dual

# Compute the analytical derivative
analytic_derivative = math.cos(x_val) + 2*x_val

print("Function value at x=1.5:", result.real)
print("Dual-based derivative:   ", dual_derivative)
print("Analytical derivative:   ", analytic_derivative)
print("Absolute error:          ", abs(dual_derivative - analytic_derivative))


Function value at x=1.5: 3.2474949866040546
Dual-based derivative:    3.070737201667703
Analytical derivative:    3.070737201667703
Absolute error:           0.0


## Using `dual_autodiff_x`

The `dual_autodiff_x` package provides an alternative implementation of dual numbers, intended to be faster than the pure Python implementation in `dual_autodiff`. While the usage is identical—offering the same class `Dual` and similar methods—`dual_autodiff_x` uses a compiled extension (e.g., via Cython) to improve performance.

Below, we will demonstrate how to use `dual_autodiff_x` just like `dual_autodiff` and then compare their performance by timing multiple derivative evaluations.


In [4]:
import timeit
import dual_autodiff_x.dual as dfx

# We'll use the same function as before: f(x) = sin(x) + x^2
def f(u):
    return u.sin() + u**2


x_df = dfx.Dual(1.5, 1.0)


x_dfx = dfx.Dual(1.5, 1.0)

# Time the evaluation of f(x) and extraction of the derivative 100000 times
iterations = 100000
time_df = timeit.timeit(lambda: f(x_df).dual, number=iterations)
time_dfx = timeit.timeit(lambda: f(x_dfx).dual, number=iterations)

print("Time using dual_autodiff (df):       ", time_df, "seconds")
print("Time using dual_autodiff_x (dfx):    ", time_dfx, "seconds")

speedup = time_df / time_dfx if time_dfx > 0 else float('inf')
print(f"Speed-up of dual_autodiff_x over df: ~{speedup:.2f}x")


Time using dual_autodiff (df):        0.04429090000000002 seconds
Time using dual_autodiff_x (dfx):     0.04536069999999981 seconds
Speed-up of dual_autodiff_x over df: ~0.98x


## Key Takeaways

- Dual numbers automatically track derivative information without the need for symbolic or numerical differentiation.
- Using a dual number `x = a + 1*ε`, evaluating a function `f(x)` provides:
  - The function value in `f(x).real`.
  - The derivative value in `f(x).dual`.

- **`dual_autodiff`** (df): A pure Python implementation that is easy to use and understand.
- **`dual_autodiff_x`** (dfx): A faster, optimized implementation of dual numbers that leverages compiled code for performance gains.

This approach makes differentiation as simple as writing the original function, eliminating the need for manually computing derivatives. Dual numbers are particularly useful for automatic differentiation in complex mathematical models. 
